In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Cell 1: Setup (run after training)
import sys
sys.path.append(str('/content/drive/MyDrive/ResearchProject'))

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

from unet_denoiser import BlindVideoDenoiserUNet
from dataloader import BlindDenoiseDataset
from model_testing import ModelTester

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Cell 2: Load your best model
checkpoint_path = '/content/drive/MyDrive/ResearchProject/checkpoints/best_model.pt'

model = BlindVideoDenoiserUNet(in_channels=9, out_channels=3, base_channels=64, num_stages=3)
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print(f"Model loaded from epoch {checkpoint['epoch']}")
print(f"Training loss: {checkpoint['train_loss']:.6f}")
print(f"Validation loss: {checkpoint['val_loss']:.6f}")

tester = ModelTester(model, device=device)

Model loaded from epoch 5
Training loss: 0.043551
Validation loss: 0.040767


In [4]:
# Cell 3: Test on a single video from DAVIS
davis_root = '/content/drive/MyDrive/ResearchProject/DAVISDataset'

# List available videos
videos = [v for v in os.listdir(davis_root) if os.path.isdir(os.path.join(davis_root, v))]
print(f"\nAvailable videos: {len(videos)}")
print("Videos:", videos[1:11])  # Show first 10

# Test on a specific video (change this to test different videos)
test_video = videos[5]  # Change index to test different videos
video_path = os.path.join(davis_root, test_video)

print(f"\n{'='*60}")
print(f"Testing on video: {test_video}")
print(f"{'='*60}")


Available videos: 150
Videos: ['weightlifting', 'water-slide', 'volleyball-beach', 'walking', 'varanus-cage', 'tuk-tuk', 'twist-dance', 'trucks-race', 'tram', 'tractor-sand']

Testing on video: varanus-cage


In [5]:
# Cell 4: Denoise all frames in the video
denoised_frames, frame_names, original_res = tester.denoise_video_folder(
    video_path,
    resize_to=(512, 512)  # Change if needed, but must be even dimensions
)

print(f"\nSuccessfully denoised {len(denoised_frames)} frames!")
print(f"Frames resized back to original: {original_res[1]}x{original_res[0]}")

Loaded 67 frames from /content/drive/MyDrive/ResearchProject/DAVISDataset/varanus-cage
Original resolution: 3840x2160
Processing resolution: 512x512
  Denoised 10/67 frames
  Denoised 20/67 frames
  Denoised 30/67 frames
  Denoised 40/67 frames
  Denoised 50/67 frames
  Denoised 60/67 frames

Successfully denoised 67 frames!
Frames resized back to original: 3840x2160


In [6]:
# Cell 5: Generate synthetic noisy and clean versions for evaluation
# (Since DAVIS has clean frames, we'll create noisy versions ourselves)
from dataloader import BlindDenoiseDataset

# Create dataset instance to use its noise function
eval_dataset = BlindDenoiseDataset(
    root_dir=davis_root,
    noise_std_range=(5, 250),
    seed=123
)

# Load clean frames
clean_frames = []
for frame_name in frame_names:
    clean_img = Image.open(os.path.join(video_path, frame_name)).convert('RGB')
    clean_frames.append(np.array(clean_img, dtype=np.float32) / 255.0)

# Create noisy versions with fixed noise level for evaluation
noise_std = 50  # You can change this (5-250 range)
noisy_frames = []

for clean_frame in clean_frames:
    noise = np.random.normal(0, noise_std, clean_frame.shape)
    noisy = np.clip(clean_frame + noise / 255.0, 0, 1)
    noisy_frames.append(noisy)

print(f"\nCreated synthetic noisy frames with noise std: {noise_std}/255")


Created synthetic noisy frames with noise std: 50/255


In [7]:
# Cell 6: Compute metrics with noise normalization
print(f"\n{'='*60}")
print(f"Computing metrics...")
print(f"{'='*60}")

# Create noisy versions with TRACKED noise levels
noise_std_255 = 50  # Noise std in 0-255 range
noisy_frames = []
noise_stds = []  # Track noise std for each frame

for clean_frame in clean_frames:
    noise = np.random.normal(0, noise_std_255, clean_frame.shape)
    noisy = np.clip(clean_frame + noise / 255.0, 0, 1)
    noisy_frames.append(noisy)
    noise_stds.append(noise_std_255)  # Track noise level

# Compute metrics WITH noise normalization
metrics = tester.compute_metrics(noisy_frames, clean_frames, denoised_frames, noise_stds=noise_stds)

print(f"\n{'='*60}")
print(f"RAW METRICS (standard, may be influenced by noise level)")
print(f"{'='*60}")
print(f"Noisy Image PSNR:     {metrics['noisy_psnr_mean']:.2f} dB")
print(f"Denoised Image PSNR:  {metrics['denoised_psnr_mean']:.2f} dB")
print(f"PSNR Improvement:     +{metrics['psnr_improvement']:.2f} dB")
print()
print(f"Noisy Image SSIM:     {metrics['noisy_ssim_mean']:.4f}")
print(f"Denoised Image SSIM:  {metrics['denoised_ssim_mean']:.4f}")
print(f"SSIM Improvement:     +{metrics['ssim_improvement']:.4f}")

# Noise-normalized metrics
if 'psnr_normalized_improvement' in metrics:
    print(f"\n{'='*60}")
    print(f"NOISE-NORMALIZED METRICS (fair comparison across noise levels)")
    print(f"{'='*60}")
    print(f"Noisy Image PSNR (norm):     {metrics['noisy_psnr_normalized_mean']:.2f} dB")
    print(f"Denoised Image PSNR (norm):  {metrics['denoised_psnr_normalized_mean']:.2f} dB")
    print(f"PSNR Improvement (norm):     +{metrics['psnr_normalized_improvement']:.2f} dB")
    print()
    print(f"Noisy Image SSIM (norm):     {metrics['noisy_ssim_normalized_mean']:.4f}")
    print(f"Denoised Image SSIM (norm):  {metrics['denoised_ssim_normalized_mean']:.4f}")
    print(f"SSIM Improvement (norm):     +{metrics['ssim_normalized_improvement']:.4f}")


Computing metrics...

RAW METRICS (standard, may be influenced by noise level)
Noisy Image PSNR:     15.14 dB
Denoised Image PSNR:  22.98 dB
PSNR Improvement:     +7.85 dB

Noisy Image SSIM:     0.0973
Denoised Image SSIM:  0.7761
SSIM Improvement:     +0.6788

NOISE-NORMALIZED METRICS (fair comparison across noise levels)
Noisy Image PSNR (norm):     29.29 dB
Denoised Image PSNR (norm):  37.13 dB
PSNR Improvement (norm):     +7.85 dB

Noisy Image SSIM (norm):     0.8022
Denoised Image SSIM (norm):  1.2984
SSIM Improvement (norm):     +0.4962


In [8]:
# Cell 7: Visualize results on multiple frames from multiple videos
print(f"\n{'='*60}")
print(f"Visualizing results...")
print(f"{'='*60}")

# Test on multiple videos
num_videos_to_show = 6  # Change this to show results from more videos
frame_indices_per_video = [0, -1]  # First and last frame of each video

for video_idx in range(min(num_videos_to_show, len(videos))):
    video_name = videos[video_idx]
    video_path = os.path.join(davis_root, video_name)

    print(f"\n--- Video {video_idx + 1}: {video_name} ---")

    try:
        # Denoise video
        denoised_video, frame_names_video, _ = tester.denoise_video_folder(
            video_path,
            resize_to=(512, 512)
        )

        # Load clean frames
        clean_video = []
        for frame_name in frame_names_video:
            clean_img = Image.open(os.path.join(video_path, frame_name)).convert('RGB')
            clean_video.append(np.array(clean_img, dtype=np.float32) / 255.0)

        # Create noisy versions with fixed noise level
        noise_std = 50
        noisy_video = []
        for clean_frame in clean_video:
            noise = np.random.normal(0, noise_std, clean_frame.shape)
            noisy = np.clip(clean_frame + noise / 255.0, 0, 1)
            noisy_video.append(noisy)

        # Show multiple frames from this video
        num_frames_to_show = min(4, len(denoised_video))  # Show up to 4 frames per video
        frame_step = max(1, len(denoised_video) // num_frames_to_show)

        for frame_idx in range(0, len(denoised_video), frame_step):
            if frame_idx >= len(denoised_video):
                break

            clean = clean_video[frame_idx]
            noisy = noisy_video[frame_idx]
            denoised = denoised_video[frame_idx]

            # Resize for better visualization
            viz_size = (512, 512)
            clean_viz = cv2.resize(clean, (viz_size[1], viz_size[0]), interpolation=cv2.INTER_LINEAR)
            noisy_viz = cv2.resize(noisy, (viz_size[1], viz_size[0]), interpolation=cv2.INTER_LINEAR)
            denoised_viz = cv2.resize(denoised, (viz_size[1], viz_size[0]), interpolation=cv2.INTER_LINEAR)

            print(f"  Frame {frame_idx + 1}/{len(denoised_video)}")
            tester.visualize_results(clean_viz, noisy_viz, denoised_viz)

    except Exception as e:
        print(f"  Error processing video: {str(e)}")

print(f"\n{'='*60}")
print(f"Visualization complete!")
print(f"{'='*60}")


Output hidden; open in https://colab.research.google.com to view.

In [9]:
# Cell 8: Save denoised video frames (optional)
output_dir = '/content/drive/MyDrive/ResearchProject/denoised_output'
os.makedirs(output_dir, exist_ok=True)

output_video_dir = os.path.join(output_dir, test_video)
os.makedirs(output_video_dir, exist_ok=True)

for frame_name, denoised_frame in zip(frame_names, denoised_frames):
    output_img = Image.fromarray((denoised_frame * 255).astype(np.uint8))
    output_path = os.path.join(output_video_dir, frame_name)
    output_img.save(output_path)

print(f"\nDenoised frames saved to: {output_video_dir}")


Denoised frames saved to: /content/drive/MyDrive/ResearchProject/denoised_output/upside-down


In [10]:
# Cell 9: Test on multiple videos and compare
print(f"\n{'='*60}")
print(f"Testing on multiple videos...")
print(f"{'='*60}")

results = []

for video_idx, video_name in enumerate(videos[:5]):  # Test on first 5 videos
    video_path = os.path.join(davis_root, video_name)

    try:
        denoised, _ = tester.denoise_video_folder(video_path)

        # Create noisy and clean versions
        clean = []
        for f in sorted([f for f in os.listdir(video_path) if f.endswith(('.png', '.jpg', '.jpeg'))]):
            img = Image.open(os.path.join(video_path, f)).convert('RGB')
            clean.append(np.array(img, dtype=np.float32) / 255.0)

        # Generate noisy
        noisy = []
        for c in clean:
            n = np.clip(c + np.random.normal(0, 50, c.shape) / 255.0, 0, 1)
            noisy.append(n)

        # Compute metrics
        m = tester.compute_metrics(noisy, clean, denoised)
        results.append({
            'video': video_name,
            'psnr_improvement': m['psnr_improvement'],
            'ssim_improvement': m['ssim_improvement']
        })

        print(f"✓ {video_name}: PSNR +{m['psnr_improvement']:.2f} dB, SSIM +{m['ssim_improvement']:.4f}")

    except Exception as e:
        print(f"✗ {video_name}: Error - {str(e)}")



Testing on multiple videos...
Loaded 65 frames from /content/drive/MyDrive/ResearchProject/DAVISDataset/upside-down
Original resolution: 2560x1440
Processing resolution: 512x512
  Denoised 10/65 frames
  Denoised 20/65 frames
  Denoised 30/65 frames
  Denoised 40/65 frames
  Denoised 50/65 frames
  Denoised 60/65 frames
✗ upside-down: Error - too many values to unpack (expected 2)
Loaded 90 frames from /content/drive/MyDrive/ResearchProject/DAVISDataset/weightlifting
Original resolution: 3840x2160
Processing resolution: 512x512
  Denoised 10/90 frames
  Denoised 20/90 frames
  Denoised 30/90 frames
  Denoised 40/90 frames
  Denoised 50/90 frames
  Denoised 60/90 frames
  Denoised 70/90 frames
  Denoised 80/90 frames
  Denoised 90/90 frames
✗ weightlifting: Error - too many values to unpack (expected 2)
Loaded 88 frames from /content/drive/MyDrive/ResearchProject/DAVISDataset/water-slide
Original resolution: 1280x720
Processing resolution: 512x512
  Denoised 10/88 frames
  Denoised 20/

In [11]:
# Cell 10: Summary statistics
if results:
    print(f"\n{'='*60}")
    print(f"Summary Statistics (across {len(results)} videos):")
    print(f"{'='*60}")

    psnr_improvements = [r['psnr_improvement'] for r in results]
    ssim_improvements = [r['ssim_improvement'] for r in results]

    print(f"Average PSNR Improvement: +{np.mean(psnr_improvements):.2f} dB")
    print(f"Std Dev PSNR:             {np.std(psnr_improvements):.2f} dB")
    print(f"Min/Max PSNR:             {np.min(psnr_improvements):.2f} / {np.max(psnr_improvements):.2f} dB")
    print()
    print(f"Average SSIM Improvement: +{np.mean(ssim_improvements):.4f}")
    print(f"Std Dev SSIM:             {np.std(ssim_improvements):.4f}")
    print(f"Min/Max SSIM:             {np.min(ssim_improvements):.4f} / {np.max(ssim_improvements):.4f}")